# Git

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%%bash
# run below in terminal
eval `ssh-agent -s`
ssh-add /content/drive/MyDrive/github/colab
ssh -T git@github.com -y
git config --global user.email "haogeh@andrew.cmu.edu"
git config --global user.name "haogeh"

Agent pid 2555


Identity added: /content/drive/MyDrive/github/colab (haogeh@andrew.cmu.edu)


In [2]:
!cd /content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare

# Setup

In [3]:
pip install openai==1.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [4]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.4 MB/s eta 0:00:00


In [5]:
import openai
from openai import OpenAI

print(openai.__version__) # version - 1.12.0


1.12.0


### storing real credentials.json inside a google drive folder is not secure and not best practice
### please download the folder, create a github repo, then create a .gitignore

In [6]:
cd /content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare

/content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare


In [7]:
ls

credentials.json  data_3.json  examples/         README.md
data_2.json       data.json    instructions.txt  starter_code.ipynb


# Load data and credentials

In [9]:
import json

# Update the path if your file is in a different location
file_path = 'data.json'

with open(file_path, 'r') as file:
    patient_data = json.load(file)

# To verify, print a part of the data
print(patient_data['patient_demographics'])



{'name': 'John Doe', 'age': 70, 'gender': 'Male', 'admission_date': '2024-02-10', 'discharge_date': '2024-02-14'}


In [10]:
# You should not store API keys in a google drive folder - it is not secure. If using github, add credentials.json to .gitignore.

credentials_path = '/content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare/credentials.json'

with open(credentials_path, 'r') as file:
    credentials = json.load(file)

openai_api_key = credentials['openai_api_key']
umls_api_key = credentials['umls_api_key']

# Use the API key as needed, for example, to configure OpenAI's API client
openai.api_key = openai_api_key


# Explore data

In [11]:
print(f'Data type: {type(patient_data)}')

Data type: <class 'dict'>


In [12]:
print(f'Keys in data: {(patient_data.keys())}')

Keys in data: dict_keys(['patient_id', 'patient_demographics', 'diagnoses', 'drg', 'encounters', 'flowsheets', 'imaging', 'labs', 'med_orders', 'notes'])


In [13]:
# Look at encounters data, there are multiple
patient_data['encounters']

[{'date': '2024-02-10',
  'type': 'Admission',
  'reason': 'Cough, shortness of breath, hemoptysis, fever'},
 {'date': '2024-02-14', 'type': 'Discharge'}]

In [14]:
def del_sensitive_data(data,sensitive_key = ['name','firstname','lastname','patient_id']):
  keys = list(data.keys())
  for key in keys:
    if type(data[key]) is dict:
      del_sensitive_data(data[key])
    else:
      if key.lower() in sensitive_key:
        del data[key]

In [15]:
llm_patient_data = patient_data.copy()
del_sensitive_data(llm_patient_data)
llm_patient_data

{'patient_demographics': {'age': 70,
  'gender': 'Male',
  'admission_date': '2024-02-10',
  'discharge_date': '2024-02-14'},
 'diagnoses': [{'date': '2024-02-10',
   'diagnosis_code': 'J18.1',
   'description': 'Lobar pneumonia, unspecified organism'}],
 'drg': {'code': '193',
  'description': 'Simple pneumonia and pleurisy with MCC'},
 'encounters': [{'date': '2024-02-10',
   'type': 'Admission',
   'reason': 'Cough, shortness of breath, hemoptysis, fever'},
  {'date': '2024-02-14', 'type': 'Discharge'}],
 'flowsheets': [{'date': '2024-02-10',
   'time': '08:00',
   'temperature': '38.5°C',
   'heart_rate': '90 bpm',
   'blood_pressure': '130/85 mmHg',
   'respiratory_rate': '20 breaths/min',
   'oxygen_saturation': '92%'},
  {'date': '2024-02-11',
   'time': '08:00',
   'temperature': '37.8°C',
   'heart_rate': '85 bpm',
   'blood_pressure': '125/80 mmHg',
   'respiratory_rate': '18 breaths/min',
   'oxygen_saturation': '94%'},
  {'date': '2024-02-12',
   'time': '08:00',
   'temper

# Logger

In [54]:
import logging

#Setup the logger
logger = logging.getLogger('logger')  # Create a logger object
logger.setLevel(logging.DEBUG)  # Set the minimum logging level

#Create a file handler that logs even debug messages
file_handler = logging.FileHandler('llm.log')
file_handler.setLevel(logging.DEBUG)

# Create a formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
file_handler.setFormatter(formatter)

# Add the file handler to the logger
logger.addHandler(file_handler)


# Chain-of-thought solution


In [16]:
client = OpenAI(api_key=openai_api_key)

In [32]:
def ok_for_discharge(patient_data):
    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    message =  {
                "role": "user",
                "content": f"""w
                              You are a physician deciding whether to discharge a patient. Please decide whether the given patient is safe to discharge.
                              Consider all information provided, particulary the doctors' notes.
                              Write the output in this format: Yes/No
                              Data: {patient_data}
                """,
            }
    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
          message
        ],
        model="gpt-3.5-turbo",
    )
    result = chat_completion.choices[0].message.content
    logger.info(f'Q: {message}')
    logger.info(f'A: {result}')
    # Return the content of the generated message
    return result

In [33]:
def generate_patient_summary(patient_data, additional_prompts='', instructions='',example=''):
    """
    Generates a patient summary using OpenAI's GPT-3.5.

    Parameters:
    - patient_data (str): The patient data to generate the summary from.
    - additional_prompts (str): Any additional prompts

    Returns:
    - str: The generated patient summary.
    """

    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    message =  {
                "role": "user",
                "content": f"""
                              You are a physician writing a discharge letter for a patient. Use patient data from context only. Minimize hallucinations.
                              {additional_prompts}
                              Write the output in this format: {instructions}
                              Example: {example}
                              Data: {patient_data}
                """,
            }
    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
          message
        ],
        model="gpt-3.5-turbo",
    )
    result = chat_completion.choices[0].message.content
    logger.info(f'Q: {message}')
    logger.info(f'A: {result}')
    # Return the content of the generated message
    return result

In [34]:
#Instructions for a good discharge summary :

# Define the path to your file
inst_path = '/content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare/instructions.txt'
eg_path = '/content/drive/MyDrive/Colab_Notebooks/90835/LLM_Healthcare/examples/eg1.txt'


with open(inst_path, 'r') as file:
    instructions = file.read()
    print(instructions)

with open(eg_path, 'r') as file:
    example = file.read()
    print(example)

Step 1
Patient Identification: Begin by filling in the patient's essential information, like age, sex, and date of birth.

Step 2
Admission and Discharge Dates: Include the dates the patient was admitted and discharged.

Step 3
Reason for Admission: Detail the primary diagnosis or condition that necessitated hospitalization.

Step 4 
Past Medical History:Important concurrent diagnoses (e.g., CHF with specific ejection fraction, history of myocardial infarctions, significant lesions, HIV status)

Step 5
Procedures Performed: Describe any surgical or non-surgical procedures done during the hospital stay.

Step 6
Medications Prescribed: List the medications prescribed to the patient at discharge, including dosages and frequency.

Step 7
Patient's Condition at Discharge: Provide a summary of the patient's health status at discharge.

Step 8
Follow-up Care Instructions: Offer detailed instructions for post-hospital care, including medications, dietary restrictions, physical activity recomme

In [35]:
print(ok_for_discharge(llm_patient_data))

INFO:my_logger:Q: {'role': 'user', 'content': 'w\n                              You are a physician deciding whether to discharge a patient. Please decide whether the given patient is safe to discharge.\n                              Consider all information provided, particulary the doctors\' notes.\n                              Write the output in this format: Yes/No\n                              Data: {\'patient_demographics\': {\'age\': 70, \'gender\': \'Male\', \'admission_date\': \'2024-02-10\', \'discharge_date\': \'2024-02-14\'}, \'diagnoses\': [{\'date\': \'2024-02-10\', \'diagnosis_code\': \'J18.1\', \'description\': \'Lobar pneumonia, unspecified organism\'}], \'drg\': {\'code\': \'193\', \'description\': \'Simple pneumonia and pleurisy with MCC\'}, \'encounters\': [{\'date\': \'2024-02-10\', \'type\': \'Admission\', \'reason\': \'Cough, shortness of breath, hemoptysis, fever\'}, {\'date\': \'2024-02-14\', \'type\': \'Discharge\'}], \'flowsheets\': [{\'date\': \'2024-02-10

Yes


In [36]:
additional_prompts='Focus on precision and minimize any assumptions or hallucinations not directly supported by the data. '
print(generate_patient_summary(llm_patient_data,additional_prompts,instructions))

INFO:my_logger:Q: {'role': 'user', 'content': '\n                              You are a physician writing a discharge letter for a patient. Use patient data from context only. Minimize hallucinations.\n                              Focus on precision and minimize any assumptions or hallucinations not directly supported by the data. \n                              Write the output in this format: Step 1\nPatient Identification: Begin by filling in the patient\'s essential information, like age, sex, and date of birth.\n\nStep 2\nAdmission and Discharge Dates: Include the dates the patient was admitted and discharged.\n\nStep 3\nReason for Admission: Detail the primary diagnosis or condition that necessitated hospitalization.\n\nStep 4 \nPast Medical History:Important concurrent diagnoses (e.g., CHF with specific ejection fraction, history of myocardial infarctions, significant lesions, HIV status)\n\nStep 5\nProcedures Performed: Describe any surgical or non-surgical procedures done du

Step 1
Patient Identification:
- Age: Data indicates that the patient is 70 years old.
- Gender: Male

Step 2
Admission and Discharge Dates:
- The patient was admitted on February 10, 2024.
- Discharged on February 14, 2024.

Step 3
Reason for Admission:
- The patient was admitted with symptoms indicative of pneumonia.

Step 4
Past Medical History:
- No significant past medical history provided in the data.

Step 5
Procedures Performed:
- No surgical or non-surgical procedures were mentioned in the data.

Step 6
Medications Prescribed:
- Amoxicillin IV 500 mg, every 8 hours initially; then 250 mg daily orally.
- Paracetamol as needed.
- Atorvastatin PO 20 mg nightly throughout the stay.

Step 7
Patient's Condition at Discharge:
- The patient's condition improved with treatment.
- Medically fit for discharge with instructions for oral antibiotics for 5 more days.

Step 8
Follow-up Care Instructions:
- Patient instructed to continue oral antibiotics for 5 more days.
- Follow-up appointme

# RAG

In [45]:
def generate_rag_query(patient_data):
    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    message = {
                "role": "user",
                "content": f"""
                              Suppose you are a physician writing a discharge letter for a patient. You will include a short paragraph explaning the disease for the patient.
                              You will use the Unified Medical Language System (UMLS) for retrieval augmented generation.
                              Based on the patient data, generate a query for the UMLS.
                              Write the output in this format: one word for the disease
                              Example: Diabete
                              Data: {patient_data}
                """,
    }

    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
          message
        ],
        model="gpt-3.5-turbo",
    )
    result = chat_completion.choices[0].message.content
    logger.info(f'Q: {message}')
    logger.info(f'A: {result}')
    # Return the content of the generated message
    return result

In [46]:
import requests

def search_umls(term, api_key):
    search_url = f"https://uts-ws.nlm.nih.gov/rest/search/current"
    params = {
        'apiKey': api_key,
        'string':f'{term}'
    }
    response = requests.get(search_url, params=params)
    logger.info(f'Search UMLS: term: {term}')
    return response.json()

api_key = umls_api_key

In [47]:
def get_concept_definition(cui, api_key):
    """Get the concept definition using the CUI and service ticket."""
    api_url = f"https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{cui}/definitions"
    params = {
        'apiKey': api_key,
    }
    response = requests.get(api_url, params=params)
    logger.info(f'''
    Get Concept Definition: cui: {cui}
    Response: {response.json()}
                ''')
    return response.json()

In [48]:
def generate_rag_explanation(patient_data,umls):
    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    message =   {
                "role": "user",
                "content": f"""
                              Suppose you are a physician writing a discharge letter for a patient. You will include a short paragraph explaning the disease for the patient.
                              You will use the Unified Medical Language System (UMLS) for retrieval augmented generation.
                              Based on the patient data and retrieved UMLS results, generate a brief explanation (100-200 words) of the disease for the patient.
                              Do not use unnecessary medical terminology to ensure the patient can understand the disease.
                              Data: {patient_data}
                              UMLS result: {umls}
                              Format:
                                Terminology:
                                Explanation:
                """,
    }
    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
          message
        ],
        model="gpt-3.5-turbo",
    )
    result = chat_completion.choices[0].message.content
    logger.info(f'Q: {message}')
    logger.info(f'A: {result}')
    # Return the content of the generated message
    return result

In [49]:
query = generate_rag_query(llm_patient_data)
print(query)

INFO:my_logger:Q: {'role': 'user', 'content': '\n                              Suppose you are a physician writing a discharge letter for a patient. You will include a short paragraph explaning the disease for the patient.\n                              You will use the Unified Medical Language System (UMLS) for retrieval augmented generation.\n                              Based on the patient data, generate a query for the UMLS.\n                              Write the output in this format: one word for the disease\n                              Example: Diabete\n                              Data: {\'patient_demographics\': {\'age\': 70, \'gender\': \'Male\', \'admission_date\': \'2024-02-10\', \'discharge_date\': \'2024-02-14\'}, \'diagnoses\': [{\'date\': \'2024-02-10\', \'diagnosis_code\': \'J18.1\', \'description\': \'Lobar pneumonia, unspecified organism\'}], \'drg\': {\'code\': \'193\', \'description\': \'Simple pneumonia and pleurisy with MCC\'}, \'encounters\': [{\'date\': 

Pneumonia


In [50]:
results = search_umls(query, api_key)
print(results)
cui = None
if results['result']['results']:
  cui = results['result']['results'][0]['ui']
print(cui)

INFO:my_logger:Search UMLS: term: Pneumonia


{'pageSize': 25, 'pageNumber': 1, 'result': {'classType': 'searchResults', 'results': [{'ui': 'C0032285', 'rootSource': 'MTH', 'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0032285', 'name': 'Pneumonia'}, {'ui': 'C0155862', 'rootSource': 'MTH', 'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0155862', 'name': 'Streptococcal pneumonia'}, {'ui': 'C0032290', 'rootSource': 'MTH', 'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0032290', 'name': 'Aspiration Pneumonia'}, {'ui': 'C0206061', 'rootSource': 'MTH', 'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0206061', 'name': 'Pneumonia, Interstitial'}, {'ui': 'C0032302', 'rootSource': 'MTH', 'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0032302', 'name': 'Mycoplasma pneumonia'}, {'ui': 'C0032310', 'rootSource': 'MSH', 'uri': 'https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0032310', 'name': 'Pneumonia, Viral'}, {'ui': 'C0032308', 'rootSource': 'MTH', 'uri': 'https://uts-ws.n

In [51]:
if cui:
  results = get_concept_definition(cui, api_key)
  print(results)
definition = results['result']

INFO:my_logger:
    Get Concept Definition: cui: C0032285
    Response: {'pageSize': 25, 'pageNumber': 1, 'pageCount': 1, 'result': [{'rootSource': 'MSHCZE', 'value': 'Infekce plic, často doprovázená zánětem.', 'classType': 'Definition', 'sourceOriginated': True}, {'rootSource': 'CSP', 'value': 'inflammation of the lungs with consolidation and exudation.', 'classType': 'Definition', 'sourceOriginated': True}, {'rootSource': 'MSHSWE', 'value': 'Infektion i lungorna som ofta åtföljs av en inflammation.', 'classType': 'Definition', 'sourceOriginated': True}, {'rootSource': 'MSH', 'value': 'Infection of the lung often accompanied by inflammation.', 'classType': 'Definition', 'sourceOriginated': True}, {'rootSource': 'MEDLINEPLUS', 'value': '<h3>What is pneumonia?</h3> <p>Pneumonia is an infection in one or both of the lungs. It causes the air sacs of the lungs to fill up with fluid or pus. It can range from mild to severe, depending on the type of germ causing the infection, your age, and 

{'pageSize': 25, 'pageNumber': 1, 'pageCount': 1, 'result': [{'rootSource': 'MSHCZE', 'value': 'Infekce plic, často doprovázená zánětem.', 'classType': 'Definition', 'sourceOriginated': True}, {'rootSource': 'CSP', 'value': 'inflammation of the lungs with consolidation and exudation.', 'classType': 'Definition', 'sourceOriginated': True}, {'rootSource': 'MSHSWE', 'value': 'Infektion i lungorna som ofta åtföljs av en inflammation.', 'classType': 'Definition', 'sourceOriginated': True}, {'rootSource': 'MSH', 'value': 'Infection of the lung often accompanied by inflammation.', 'classType': 'Definition', 'sourceOriginated': True}, {'rootSource': 'MEDLINEPLUS', 'value': '<h3>What is pneumonia?</h3> <p>Pneumonia is an infection in one or both of the lungs. It causes the air sacs of the lungs to fill up with fluid or pus. It can range from mild to severe, depending on the type of germ causing the infection, your age, and your overall health.</p> <h3>What causes pneumonia?</h3> <p><a href="htt

In [52]:
print(generate_rag_explanation(llm_patient_data,definition))

INFO:my_logger:Q: {'role': 'user', 'content': '\n                              Suppose you are a physician writing a discharge letter for a patient. You will include a short paragraph explaning the disease for the patient.\n                              You will use the Unified Medical Language System (UMLS) for retrieval augmented generation.\n                              Based on the patient data and retrieved UMLS results, generate a brief explanation (100-200 words) of the disease for the patient.\n                              Do not use unnecessary medical terminology to ensure the patient can understand the disease.\n                              Data: {\'patient_demographics\': {\'age\': 70, \'gender\': \'Male\', \'admission_date\': \'2024-02-10\', \'discharge_date\': \'2024-02-14\'}, \'diagnoses\': [{\'date\': \'2024-02-10\', \'diagnosis_code\': \'J18.1\', \'description\': \'Lobar pneumonia, unspecified organism\'}], \'drg\': {\'code\': \'193\', \'description\': \'Simple pneu

Terminology: Pneumonia

Explanation: Pneumonia is an infection in one or both of the lungs that causes the air sacs to fill up with fluid or pus. It can vary in severity depending on the type of germ causing the infection, your age, and overall health. In most cases, pneumonia is caused by bacteria, leading to symptoms like fever, chills, cough with phlegm, shortness of breath, chest pain, nausea, and vomiting. Other causes can include viruses and fungi, especially in individuals with weakened immune systems. Diagnosis involves a physical exam, chest X-ray, blood tests, and sometimes additional imaging or fluid analysis. Treatment typically includes antibiotics for bacterial pneumonia, antivirals for viral pneumonia, and antifungals for fungal pneumonia. Recovery time can vary, and prevention measures like vaccines and good hygiene practices are recommended to reduce the risk of pneumonia.


## Close Logger

In [53]:
file_handler.close()
logger.removeHandler(file_handler)